# 1.01 Deep Particle Crossing Locations

---

Author: Riley X. Brady

Date: 11/18/2020

This notebook pulls in the 19,002 particles that have been identified as those who last cross 1000 m in the ACC (S of 45S and outside of the annual sea ice zone) and finds the x, y location in which they last upwell at 1000 m. These locations are used in subsequent notebooks for filtering and visualization.

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr

from dask.distributed import Client

In [2]:
print(f"xarray: {xr.__version__}")
print(f"numpy: {np.__version__}")

xarray: 0.16.1
numpy: 1.19.4


In [3]:
# This is my TCP client from the `launch_cluster` notebook. I use it
# for distributed computing with `dask` on NCAR's machine, Casper.
client = Client("tcp://...")

In [4]:
# Load in the `zarr` file, which is pre-chunked and already has been
# filtered from the original 1,000,000 particles to the 19,002 that
# upwell last across 1000 m S of 45S and outside of the annual sea ice
# edge.
filepath = "../data/southern_ocean_deep_upwelling_particles.zarr/"
ds = xr.open_zarr(filepath, consolidated=True)

In [5]:
def _compute_idx_of_last_1000m_crossing(z):
    """Find index of final time particle upwells across 1000 m.

    z : zLevelParticle
    """
    currentDepth = z
    previousDepth = np.roll(z, 1)
    previousDepth[0] = 999  # So we're not dealing with a nan here.
    cond = (currentDepth >= -1000) & (previousDepth < -1000)
    idx = (
        len(cond) - np.flip(cond).argmax() - 1
    )  # Finds last location that condition is true.
    return idx


def compute_xy_of_last_crossing(x, y, z):
    """Convert crossing point into x, y coordinates.

    x : lonParticle (radians)
    y : latParticle (radians)
    z : zLevelParticle (m)
    """
    idx = _compute_idx_of_last_1000m_crossing(z)
    lon = np.rad2deg(x[idx])
    lat = np.rad2deg(y[idx])
    return np.array([lon, lat])

In [6]:
ds = ds.chunk({"time": -1, "nParticles": 6000})
x = ds.lonParticle.persist()
y = ds.latParticle.persist()
z = ds.zLevelParticle.persist()

In [7]:
result = xr.apply_ufunc(
    compute_xy_of_last_crossing,
    x,
    y,
    z,
    input_core_dims=[["time"], ["time"], ["time"]],
    vectorize=True,
    dask="parallelized",
    output_core_dims=[["coordinate"]],
    output_dtypes=[float],
    dask_gufunc_kwargs={"output_sizes": {"coordinate": 2}},
)

In [8]:
%time crossings = result.compute()
crossings = crossings.assign_coords(coordinate=["x", "y"])

CPU times: user 12.4 ms, sys: 4.16 ms, total: 16.5 ms
Wall time: 1.29 s


In [9]:
ds = xr.Dataset()
ds["lon_crossing"] = crossings.sel(coordinate="x")
ds["lat_crossing"] = crossings.sel(coordinate="y")
ds.attrs[
    "description"
] = "x/y locations of *final* 1000m crossing for particles that occur < 45S; reach 200 m following this crossing; and happen outside of the 75% annual sea ice zone."
ds.to_netcdf("../data/postproc/1000m.crossing.locations.nc")